In [2]:
import numpy as np
import math
import random
import timeit
from scipy.special import expit

data = np.loadtxt('heart.dat')

for k in range(len(data)):
    if data[k][13]==2:
        data[k][13]=1 
    else:
        data[k][13]=-1
        
data_x = data[:220]
data_t = data[220:]
dtype_ = np.dtype(dtype="float64")

input_data=[]

for r in range(len(data_x)):
    input_data.append([data_x[r][:12],data_x[r][13:]])

testing_data=[]

for s in range(len(data_t)):
    testing_data.append([data_t[s][:12],data_t[s][13:]])

#Setting up the data type for numpy arrays
dtype="float64"
dtype_ = np.dtype(dtype)
x_i = list()
y_i = list()
for i, j in input_data:
        x_i.append(i)
        y_i.append(j)
x_i, y_i = np.array(x_i, dtype=dtype_), np.array(y_i, dtype=dtype_)

#Initializing the values of the Omega and Beta
start_data_w= np.random.randint(np.size(x_i[0]), size=np.size(x_i[0]))
start_data_b = np.random.randint(np.size(y_i))

#Gradient function
values = []
def gradient(training,W,B):
    L = 0.0001
    m = np.size(W)
    a = np.zeros(1)
    s = np.zeros(m+1)
    b = np.append(W,a)

    G0 = 2*L*b
    
    for x,y in input_data:
        c = np.array(np.sum(-y*x), dtype=dtype)
        d = np.array([[-y]], dtype=dtype)
        e = np.array([[c],d], dtype=dtype)

        if np.add(1,np.sum(-y*np.add(np.sum(W*x),B)))<=0:
            values.append([[np.add(G0,s)]])

        elif np.add(1,np.sum(-y*np.add(np.sum(W*x),B)))>0:
            values.append([[np.add(G0,e)]])

def ssgm(gradient, training_d, start_w, start_b, stepsize, tolerance, n_iter, batch_size,
        dtype="float64", random_state=None):
    
    start = timeit.default_timer()
    
    #Checking if the gradient is callable
    if not callable(gradient):
        raise TypeError("'gradient' must be callable")

    #Setting up the data type for numpy arrays
    dtype_ = np.dtype(dtype)

    #Converting x and y to numpy arrays
    n_obs = x_i.shape[0]
    if n_obs != y_i.shape[0]:
        raise ValueError("'x' and 'y' lengths do not match")
    x_iy_i = np.c_[x_i.reshape(n_obs, -1), y_i.reshape(n_obs, 1)]
    
    #Initializing the random number generator
    seed = None if random_state is None else int(random_state)
    rng = np.random.default_rng(seed=seed)

    #Initializing the values of the variables
    StartingV = [start_w, start_b]
    vector = np.array(start_w, dtype=dtype_)
    beta = np.array(start_b, dtype=dtype_)

    #Setting up and checking the stepsize
    stepsize = np.array(stepsize, dtype=dtype_)
    
    if np.any(stepsize <= 0):
        raise ValueError("'stepsize' must be greater than zero")

    #Setting up and checking the size of minibatches
    batch_size = int(batch_size)
    if not 0 < batch_size <= n_obs:
        raise ValueError(
            "'batch_size' must be greater than zero and less than "
            "or equal to the number of observations"
        )

    #Setting up and checking the maximal number of iterations
    n_iter = int(n_iter)
    if n_iter <= 0:
        raise ValueError("'n_iter' must be greater than zero")

    #Setting up and checking the tolerance
    tolerance = np.array(tolerance, dtype=dtype_)
    if np.any(tolerance <= 0):
        raise ValueError("'tolerance' must be greater than zero")
    
    #Performing the gradient descent loop
    for _ in range(n_iter):
        #Shuffle x and y
        rng.shuffle(x_iy_i)

        #Performing minibatch moves
        for StartingV in range(0, n_obs, batch_size):
            stop_t = StartingV + batch_size
            x_batch, y_batch = x_iy_i[StartingV:stop_t, :-1], x_iy_i[StartingV:stop_t, -1:]

            #Recalculating the difference
            diff = np.sum(-stepsize,gradient([x_batch, y_batch], vector, beta))

            #Checking if the absolute difference is small enough
            if (abs(diff.all()) > tolerance):
                #Updating the values of the variables
                vector+=diff
                beta+=diff
            else:
                print("break",start,stop)
                break
                
        print(vector,beta)           
    count_accuracy = 0
    
    for h,l in testing_data:
        if np.add(np.sum(np.array(vector)*np.array(h)),np.array(beta))>=0 and np.array(l)==1:
            count_accuracy += 1

        elif np.add(np.sum(np.array(vector)*np.array(h)),np.array(beta))<0 and np.array(l)==-1:
            count_accuracy += 1
            
        check_accuracy = np.divide(count_accuracy,len(testing_data))*100
    
    stop = timeit.default_timer()
    
    return(vector if vector.shape else vector.item(), 
           print("The local minimum occurs at: Omega={}".format(vector), "Beta={}".format(beta)),
          print("Reported accuracy:{}%".format(check_accuracy)), print('Run time is: ', stop-start))

print(ssgm(gradient, input_data, start_data_w, start_data_b, 0.0001, 1e-04, 100, 1))

[ 2.978 10.978  0.978  7.978  3.978  1.978  4.978  1.978  0.978 -0.022
  8.978 10.978] 203.97799999999927
[ 2.956 10.956  0.956  7.956  3.956  1.956  4.956  1.956  0.956 -0.044
  8.956 10.956] 203.95599999999854
[ 2.934 10.934  0.934  7.934  3.934  1.934  4.934  1.934  0.934 -0.066
  8.934 10.934] 203.9339999999978
[ 2.912 10.912  0.912  7.912  3.912  1.912  4.912  1.912  0.912 -0.088
  8.912 10.912] 203.91199999999708
[ 2.89 10.89  0.89  7.89  3.89  1.89  4.89  1.89  0.89 -0.11  8.89 10.89] 203.88999999999635
[ 2.868 10.868  0.868  7.868  3.868  1.868  4.868  1.868  0.868 -0.132
  8.868 10.868] 203.86799999999562
[ 2.846 10.846  0.846  7.846  3.846  1.846  4.846  1.846  0.846 -0.154
  8.846 10.846] 203.8459999999949
[ 2.824 10.824  0.824  7.824  3.824  1.824  4.824  1.824  0.824 -0.176
  8.824 10.824] 203.82399999999416
[ 2.802 10.802  0.802  7.802  3.802  1.802  4.802  1.802  0.802 -0.198
  8.802 10.802] 203.80199999999343
[ 2.78 10.78  0.78  7.78  3.78  1.78  4.78  1.78  0.78 -0.22 